<h1>Automated Gating with Immunova: Creating a template for PDMC samples</h1>

In [1]:
from immunova.data.project import Project
from immunova.data.mongo_setup import global_init
from immunova.flow.gating.actions import Template
from tqdm import tqdm_notebook, tqdm
from functools import partial
import pandas as pd
global_init()

In [3]:
peritonitis = Project.objects(project_id='Peritonitis').get()

In [4]:
peritonitis.list_fcs_experiments()

['PBMC_T', 'PDMC_T', 'PBMC_M', 'PDMC_M']